In [ ]:
%matplotlib inline

import os
import cmaps
import salem
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
from pylab import rcParams
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from myfunc import timer
from myfunc import DirMan
import config
import matplotlib.font_manager as fm

# configuration
# resolution = "0p1"
resolution = "500"
region = [-180,180,-60,90]
data_path = f'/tera04/zhwei/xionghui/bedrock/run/{resolution}/'
post_data_path = '/tera04/zhwei/xionghui/bedrock/'
shp_path = '/tera04/zhwei/xionghui/bedrock/Shp/'
fig_path = f'/home/xuxh22/stu01/Bedrock/fig/{resolution}/'
path = '/home/xuxh22/stu01/Bedrock/'
if resolution == "0p1":
    size = 0.1
elif resolution == "500":
    size = 0.0005

print('python draw_g2_scatter.py')
print(data_path)
dir_man = DirMan(data_path)
dir_man.enter()
os.makedirs(f'{fig_path}/global_map_2', exist_ok=True)

shp = gpd.GeoDataFrame.from_file(shp_path+'World_CN/ne_10m_admin_0_countries_chn.shp')

pd.set_option('display.max_columns', None)
font = {'family': 'Times New Roman'}
matplotlib.rc('font', **font)

params = {'backend': 'ps',
          'axes.labelsize': 25,
          'grid.linewidth': 0.2,
          'font.size': 25,
          'legend.fontsize': 18,
          'legend.frameon': False,
          'xtick.labelsize': 30,
          'xtick.direction': 'out',
          'ytick.labelsize': 30,
          'ytick.direction': 'out',
          'legend.handlelength': 1,
          'legend.handleheight': 1,
          'savefig.bbox': 'tight',
          'axes.unicode_minus': False,
          "mathtext.default":"regular",
          'text.usetex': False}
rcParams.update(params)

font_properties = fm.FontProperties(weight='bold')

casename1 = 'bedrock_1'
path1 = f'{data_path}/cases/{casename1}/'
casename2 = 'bedrock_2'
path2 = f'{data_path}/cases/{casename2}/'
casename3 = 'bedrock_3'
path3 = f'{data_path}/cases/{casename3}/'

In [ ]:
def draw1(df1,name,level,cmap):
    df1['diff'] = df1[name[0]] - df1[name[1]]

    fig = plt.figure(figsize=(12, 6), dpi=500)

    fig.subplots_adjust(left=0, right=1, 
                    bottom=0, top=1) 
        
    #Create a subgraph grid with 2 rows and 3 columns
    gs = GridSpec(2, 6)
    ax = fig.add_subplot(gs[:, :], projection=ccrs.PlateCarree())

    # Set drawing mode(note:extent's lat from positive to negative)
    img = ax.scatter(df1['lon'], df1['lat'], c=df1['diff'], 
                    s=size, linewidths=0, edgecolors="k", 
                    cmap=cmap, zorder=1, vmin=level[0], vmax=level[-1])

    for spine in ax.spines.values():
        spine.set_edgecolor('black')  
        spine.set_linewidth(0)  

    ax.set_xlim(region[0], region[1])
    ax.set_ylim(region[2], region[3])

    # coastline = cfeature.NaturalEarthFeature('physical', 'coastline', '50m', edgecolor='0.6', facecolor='none')
    rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '110m', edgecolor='0.6', facecolor='none')
    ax.add_feature(cfeature.LAND, facecolor='0.95')
    # ax.add_feature(coastline, linewidth=0.6)
    ax.add_feature(cfeature.LAKES, alpha=1, facecolor='white', edgecolor='white')
    ax.add_feature(rivers, linewidth=0.8)
    # ax.gridlines(draw_labels=False, linestyle=':', linewidth=0.7, color='grey', alpha=0.8)

    ax.add_feature(cfeature.COASTLINE)
    ax.set_extent(region)
    ax.xaxis.set_major_formatter(LongitudeFormatter())
    ax.yaxis.set_major_formatter(LatitudeFormatter())

    # From the bottom left corner x, y, width, height
    cbar_ax = fig.add_axes([0.4, 0.08, 0.4, 0.03], frameon = False) 
    cb = fig.colorbar(img, 
                    drawedges=False,
                    ticks=level, 
                    cax=cbar_ax, 
                    orientation='horizontal',
                    spacing='uniform')

    cb.ax.tick_params(labelsize=20)
    cb.ax.yaxis.set_tick_params(direction='out', width=1.5)
    for label in cb.ax.get_xticklabels() + cb.ax.get_yticklabels():
        label.set_fontproperties(font_properties)
    cb.set_label(f'{name[3]}', fontsize=25, fontweight='bold',labelpad=-92)

    plt.tight_layout()

    # plt.savefig(f"{fig_path}/global_map_2/g2_{name[2]}_diff.png")
    plt.show()

In [ ]:
df = pd.read_csv(f'{data_path}csv/Global_cases_sr.csv')
df1 = df.copy()

In [ ]:
rgb_list = ['#606060','#8ec0cb','#00CC66','#66CC00',
                                '#69aa4c','#CCCC00','#ebc874','#99004C','#FF6666']
name = [f'{path1}Sr_mean', f'{path2}Sr_mean', 'Sr_case1', '(Exp1 - Exp2) \n$S_{r}$ (mm)']
level = np.arange(-200,300,100)
cmap = cmaps.cmocean_balance
draw1(df1,name,level,cmap)

In [ ]:
name = [f'{path1}Sr_mean', f'{path3}Sr_mean', 'Sr_case1', '(Exp1 - Exp3) \n$S_{r}$ (mm)']
level = np.arange(-200,300,100)
cmap = cmaps.cmocean_balance
draw1(df1,name,level,cmap)

In [ ]:
rgb_list = ['#606060','#8ec0cb','#00CC66','#66CC00',
                                '#69aa4c','#CCCC00','#ebc874','#99004C','#FF6666']
name = [f'{path2}Sr_mean', f'{path3}Sr_mean', 'Sr_case1', '(Exp2 - Exp3) \n$S_{r}$ (mm)']
level = np.arange(-200,300,100)
cmap = cmaps.cmocean_balance
draw1(df1,name,level,cmap)

In [ ]:
# name = [f'{path1}Sr_mean', 'Sr', 'Sr_case1', '(Exp1 - our calculated) \n$S_{r}$ (mm)']
# level = np.arange(-200,300,100)
# cmap = cmaps.cmocean_curl
# draw1(df1,name,level,cmap)

In [ ]:
# name = [f'{path2}Sr_mean', 'Sr', 'Sr_case2', '(Exp2 - our calculated) \n$S_{r}$ (mm)']
# level = np.arange(-200,300,100)
# cmap = cmaps.cmocean_curl
# draw1(df1,name,level,cmap)

In [ ]:
# name = [f'{path3}Sr_mean', 'Sr', 'Sr_case3', '(Exp3 - our calculated) \n$S_{r}$ (mm)']
# level = np.arange(-200,300,100)
# cmap = cmaps.cmocean_curl
# draw1(df1,name,level,cmap)

In [ ]:
df = pd.read_csv(f'{data_path}csv/Global_cases_sb.csv')
df1 = df.copy()

In [ ]:
rgb_list = ['#606060','#8ec0cb','#00CC66','#66CC00',
                                '#69aa4c','#CCCC00','#ebc874','#99004C','#FF6666']

name = [f'{path1}Sbedrock_mean', f'{path2}Sbedrock_mean', 'Sbedrock', '(Exp1 - Exp2)\n$S_{bedrock}$ (mm)']
level = np.arange(-200,300,100)
cmap = cmaps.cmocean_balance
draw1(df1,name,level,cmap)

In [ ]:
rgb_list = ['#606060','#8ec0cb','#00CC66','#66CC00',
                                '#69aa4c','#CCCC00','#ebc874','#99004C','#FF6666']

name = [f'{path1}Sbedrock_mean', f'{path3}Sbedrock_mean', 'Sbedrock', '(Exp1 - Exp3)\n$S_{bedrock}$ (mm)']
level = np.arange(-200,300,100)
cmap = cmaps.cmocean_balance
draw1(df1,name,level,cmap)

In [ ]:
rgb_list = ['#606060','#8ec0cb','#00CC66','#66CC00',
                                '#69aa4c','#CCCC00','#ebc874','#99004C','#FF6666']

name = [f'{path2}Sbedrock_mean', f'{path3}Sbedrock_mean', 'Sbedrock', '(Exp2 - Exp3)\n$S_{bedrock}$ (mm)']
level = np.arange(-200,300,100)
cmap = cmaps.cmocean_balance
draw1(df1,name,level,cmap)

In [ ]:
df = pd.read_csv(f'{data_path}csv/Global_cases_ss.csv')
df1 = df.copy()

In [ ]:
rgb_list = ['#606060','#8ec0cb','#00CC66','#66CC00',
                                '#69aa4c','#CCCC00','#ebc874','#99004C','#FF6666']

name = [f'{path1}Ssoil_mean', f'{path2}Ssoil_mean', 'Ssoil', '(Exp1 - Exp2)\n$S_{soil}$ (mm)']
level = np.arange(-200,300,100)
cmap = cmaps.cmocean_balance
draw1(df1,name,level,cmap)

In [ ]:
rgb_list = ['#606060','#8ec0cb','#00CC66','#66CC00',
                                '#69aa4c','#CCCC00','#ebc874','#99004C','#FF6666']

name = [f'{path1}Ssoil_mean', f'{path3}Ssoil_mean', 'Ssoil', '(Exp1 - Exp3)\n$S_{soil}$ (mm)']
level = np.arange(-200,300,100)
cmap = cmaps.cmocean_balance
draw1(df1,name,level,cmap)

In [ ]:
rgb_list = ['#606060','#8ec0cb','#00CC66','#66CC00',
                                '#69aa4c','#CCCC00','#ebc874','#99004C','#FF6666']

name = [f'{path2}Ssoil_mean', f'{path3}Ssoil_mean', 'Ssoil', '(Exp2 - Exp3)\n$S_{soil}$ (mm)']
level = np.arange(-200,300,100)
cmap = cmaps.cmocean_balance
draw1(df1,name,level,cmap)

In [ ]:
df = pd.read_csv(f'{data_path}csv/Global_cases_runoff_et.csv')
df1 = df.copy()

In [ ]:
name = [f'{path1}rnof_mean', f'{path2}rnof_mean', 'Q_case1', '(Exp1 - Exp2) \ntotal runoff (mm)']
level = np.arange(-400,600,200)
cmap = cmaps.cmocean_balance
draw1(df1,name,level,cmap)

In [ ]:
name = [f'{path1}rnof_mean', f'{path3}rnof_mean', 'Q_case1', '(Exp1 - Exp3) \ntotal runoff (mm)']
level = np.arange(-400,600,200)
cmap = cmaps.cmocean_balance
draw1(df1,name,level,cmap)

In [ ]:
name = [f'{path2}rnof_mean', f'{path3}rnof_mean', 'Q_case1', '(Exp2 - Exp3) \ntotal runoff (mm)']
level = np.arange(-400,600,200)
cmap = cmaps.cmocean_balance
draw1(df1,name,level,cmap)

In [ ]:
# name = [f'{path1}rnof_mean', 'Q_mean', 'Q_case1', '(Exp1 - our calculated) \ntotal runoff (mm)']
# level = np.arange(-400,600,200)
# cmap = cmaps.cmocean_balance
# draw1(df1,name,level,cmap)

In [ ]:
# name = [f'{path2}rnof_mean', 'Q_mean', 'Q_case2', '(Exp2 - our calculated) \ntotal runoff (mm)']
# level = np.arange(-400,600,200)
# cmap = cmaps.cmocean_balance
# draw1(df1,name,level,cmap)

In [ ]:
# name = [f'{path3}rnof_mean', 'Q_mean', 'Q_case3', '(Exp3 - our calculated) \ntotal runoff (mm)']
# level = np.arange(-400,600,200)
# cmap = cmaps.cmocean_balance
# draw1(df1,name,level,cmap)

In [ ]:
name = [f'{path1}fevpa_mean', f'{path2}fevpa_mean', 'ET_case1', '(Exp1 - Exp2) \nevapotranspiration (mm)']
level = np.arange(-600,900,300)
cmap = cmaps.cmocean_balance
draw1(df1,name,level,cmap)

In [ ]:
name = [f'{path1}fevpa_mean', f'{path3}fevpa_mean', 'ET_case1', '(Exp1 - Exp3) \nevapotranspiration (mm)']
level = np.arange(-600,900,300)
cmap = cmaps.cmocean_balance
draw1(df1,name,level,cmap)

In [ ]:
name = [f'{path2}fevpa_mean', f'{path3}fevpa_mean', 'ET_case1', '(Exp2 - Exp3) \nevapotranspiration (mm)']
level = np.arange(-600,900,300)
cmap = cmaps.cmocean_balance
draw1(df1,name,level,cmap)

In [ ]:
# name = [f'{path1}fevpa_mean', 'ET_mean', 'ET_case1', '(Exp1 - our calculated) \nevapotranspiration (mm)']
# level = np.arange(-600,900,300)
# cmap = cmaps.cmocean_balance
# draw1(df1,name,level,cmap)

In [ ]:
# name = [f'{path2}fevpa_mean', 'ET_mean', 'ET_case2', '(Exp2 - our calculated) \nevapotranspiration (mm)']
# level = np.arange(-600,900,300)
# cmap = cmaps.cmocean_balance
# draw1(df1,name,level,cmap)

In [ ]:
# name = [f'{path3}fevpa_mean', 'ET_mean', 'ET_case3', '(Exp3 - our calculated) \nevapotranspiration (mm)']
# level = np.arange(-600,900,300)
# cmap = cmaps.cmocean_balance
# draw1(df1,name,level,cmap)